In [1]:
import re
import jieba
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pickle
import tensorflow as tf


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on.
file_path = '/content/drive/My Drive/McMaster/projects/NLP/Data/English-Chinese/'
list_file = 'list.txt'


In [0]:
def clean_en(text):
  text = text.lower()
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "that is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"how's", "how is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"n't", " not", text)
  text = re.sub(r"n'", "ng", text)
  text = re.sub(r"'bout", "about", text)
  text = re.sub(r"'til", "until", text)
  text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
  return text

def clean_zh(x):
  x = re.sub('[,.!?。，！？\n\t]', '', x).strip()
  # use jieba parser to cut
  #x = jieba.cut(x)
  #return ' '.join(x)
  return x


In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(file_path+list_file, 'r') as lf:
  for data_path in lf:
    with open((file_path+data_path).strip(), 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split('<s>')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t' + target_text + '\n'
        input_texts.append(clean_en(input_text))
        target_texts.append(clean_zh(target_text))
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 6000
Number of unique input tokens: 27
Number of unique output tokens: 3670
Max sequence length for inputs: 299
Max sequence length for outputs: 142


In [5]:
print(target_texts)

['为了 更好 的 锐度 但是 附带 的 会 多一些 颗粒 度 可以 使用 这个 显影剂 的 11 稀释液', '他 还 把 宣扬 自己 思想 的 所谓 绿皮书 称作 新 福音书', '微风 推着 我 去 爱抚 它 的 长耳朵', '它们 的 先烈 们 的 鲜血 是 白流 了', '最后 在 月 31 日湖 人 将 前往 汽车城 底特律 挑战 活塞队 活塞 近来 在 东部 排名 第二', '真是 天造地设 的 一对 我 父亲 喜欢 结交 名 人杰 姬 酷爱 金钱 亚历山大 在 婚礼 上 讥讽 道 他 和 克里斯蒂娜 从未 同 他们 的 继母 和睦相处 过', '2006 年 沃尔玛 的 推荐 引擎 竟 将 人猿 星球 与 马丁 · 路德 · 金 的 记录片 配成 了 一对 为此 沃尔玛 遭到 了 种族歧视 的 指控', '通过 电子探针 显微 分析 确定 贫化 渣 中 主要 铜 相为 冰铜 相', '吉姆 靠 给 人 擦皮鞋 为生', '用 甘氨酸 模拟 胶原 研究 间苯二酚 恶唑烷 E 鞣性 基质 的 形成 以及 与 胶原 之间 的 反应 特性', '免费 市内 取送 免费 提供 打印 译稿 及 电子 文档 各 一份', '基尔 大学 以 其 令人兴奋 的 方式 高等教育 美丽 的 校园 强大 的 社区 精神 和 优秀学生 的 生活', '这些 中药 中 就 有 生长 于 云南 和 贵州 的 茯苓', '或者 是因为 你 愿意 花费 无数 的 夜晚 去 安慰 他们', 'CallumOK 答案 将会 在 节目 的 最后 揭晓 到 时 我们 再 看 你 有没有 答对', '当 他 站 在 水平 的 地面 上时 后 足 爪 位于 身躯 后方 不 在 身躯 正下方 从 脚垫 到 飞节 垂直于 地面', '所以 到底 是 谁 赢 了 阿拉斯加 不算数 那 是 你们 从 俄罗斯 买 的', '在 一个 特别 寒冷 的 冬天 一对 住 在 明尼阿波利斯 市 的 夫妇 决定 去 弗罗 里 达 避寒', '热爱 温暖 的 毛 袜子 和 冰冻 柠檬汁 的 邓 布利 多变 出 柔软 舒适 的 家具', '要 成功 拿到 博士学位 你 必须 真正 的 扩展 人类 的 知识 边界', '这人 说 巴里 在 1990 年 由于 涉及 毒品 而 被 关进 监狱 在 这 以前 他 在 华盛顿 

In [21]:
print(target_characters)

['\t', '\n', ' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', 'Ñ', 'á', 'è', 'ñ', 'ö', 'ü', 'Ω', 'α', 'β', 'μ', '–', '―', '‘', '’', '•', '‧', '‰', '″', '℃', '™', '→', '∶', '⊙', '①', '②', '⑥', '⒊', '─', '△', '○', '●', '☆', '〇', '〈', '〉', '「', '」', '・', '一', '丁', '七', '万', '丈', '三', '上', '下', '不', '与', '丐', '丑', '专', '且', '世', '丘', '丙', '业', '丛', '东', '丝', '丢', '两', '严', '丧', '个', '丫', '中', '丰', '串', '临', '丸', '丹', '为', '主', '丽', '举', '乃', '久', '么', '义', '之', '乌', '乍', '乎', '乏', '乐', '乒', '乓', '乔', '乖', '乘', '乙', '九', '乞', '也', '习', '乡', '书', '买', '乱', '乳', '乸', '乾', '了', '予', '争', '事', '二', '于', '亏', '云', '互', '五', '井', '亚', '些', '亟', '亡', '亢', '交', '亥', '亦', '产', '亨', '亩', '享', '京', '亭

In [5]:
print(tf.__version__)

2.0.0-beta1


In [0]:
from nltk.translate.bleu_score import sentence_bleu

def bleu(reference, candidate):
	if len(reference) != len(candidate):
		raise ValueError('The number of sentences in both files do not match.')

	score = 0.

	for i in range(len(reference)):
		score += sentence_bleu([reference[i].strip().split()], candidate[i].strip().split())

	score /= len(reference)
	return score;

In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
#              metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy', bleu])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

RuntimeError: ignored